# Introduction

This notebook is an attemp to better understand (through implementation) the working and details of RNN implementations. We have used pytorch as a library of choice for building the RNN, but the described methods can be used with other deep learning libraries. 

**Note** that the code proviede here should only be used for practicing and understanding of the matter at hand. The offered implementation here is almost an order of magnitude slower than the one from the pytorch library. 

We heavily rely on definitions, notations and documentations from [Deep Learning](https://www.deeplearningbook.org/) book (Chp. 10) and [pytorch](https://pytorch.org/docs/stable/nn.html#rnn) documentation.

# Notation and definition of RNNs

<img src="pics/rnn1.png" style="float: left; width: 700px;">

__Notation__: The left and right of the arrow correspond to variables in domain and co-domain of the function. The object on top of the arrow represent the function that acts on the domain variable.

<img src="pics/rnn2.png" style="float: left; width: 250px;">

__Notation__: aggregate of two variables is shown as follows

<img src="pics/rnn3.png" style="float: left; width: 200px;">

Putting these together, we can represent the flow of information in a RNN cell as follows

<img src="pics/rnn4.png" style="float: left; width: 500px;">

Expanded RNNs of one layer and two layer with sequence length of 3 is shown below:

<img src="pics/rnn5.png" style="float: left; width: 700px;">

One way to do the forward pass in a multilayer RNN is to compute one layer at a time, as is shown in the following 2-layer example:

<img src="pics/animated_gif_1.gif" style="float: left; width: 700px;">

Notice how we need the initial values for the hidden variables of each layer ($h_0$ and $g_0$) to start the forward pass in that layer. Another thing to notice is that for multi-layer RNNs, the hidden variables of the lower layers act as inputs for updating the hidden variables of the layer top, the same way that $x_i$s are used for computing of the hidden variables of the first layer.

# Implementation

We implement a subset of what is implemented in pytorch:
- Only uni-directional RNN is implemented 
- We only implement the `batch_first = True` variation. 
- Only `tanh` activation is considered (in pytorch one can choose between `tanh` and `relu`).
- For automatic differentiation pytorch's `autograd` module is used. We only care about the soundness of forward pass in our implementation and delegate the differentiation to pytorch.

We use the same variable namings of parameters for compatibility with pytorch `nn.RNN` class. The `__init__` method is largely based (and copied) from pytorch. The forward method, which is the main point of this implementaion, is written form scratch. The initialization of the the object, input and output of the forward pass are all compatible with `nn.RNN` (in `batch_first=True` mode).

In [181]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch import tensor, stack
from functools import partial

In [215]:
class RNN(nn.Module):
    '''An RNN class based on pytorch implementation.'''
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # We name the variables identical to their counter parts in
        # pytorch. Please refer to pytorch source code for more
        self.ws_ih = []
        self.ws_hh = []
        self.bs_ih = []
        self.bs_hh = []
        for layer in range(num_layers):
            layer_input_size = input_size if layer == 0 else hidden_size
            w_ih = self._init_weight_((hidden_size, layer_input_size,))
            w_hh = self._init_weight_((hidden_size, hidden_size,))
            b_ih = self._init_weight_((hidden_size,))
            b_hh = self._init_weight_((hidden_size,))
            
            param_names = ['weight_ih_l{}', 'weight_hh_l{}',
                           'bias_ih_l{}', 'bias_hh_l{}']
            param_names = [name.format(layer) for name in param_names]
            layer_params = (w_ih, w_hh, b_ih, b_hh)
            
            for param, name in zip(layer_params, param_names):
                setattr(self, name, param)
            self.ws_ih.append(w_ih)
            self.ws_hh.append(w_hh)
            self.bs_ih.append(b_ih)
            self.bs_hh.append(b_hh)
                
        
    def _init_weight_(self, shape):
        k_sqrt = torch.sqrt(torch.tensor(1/self.hidden_size))
        weight = 2.0 * k_sqrt * torch.rand(shape) - k_sqrt
        return nn.Parameter(weight)
    
    
    def batch_forward(self, x, h_0):
        return self.layers_forward(self.tanh_cell, x, h_0)
                
    def cum_map(self, f, xs, h):
        # f, [x_1,x_2, ..., x_L], h_0 => [h_1=f(x_1, h_0), h_2=f(x_2, h_1), ..., h_L=f(x_L, h_{L-1})], h_L
        cum_results = []
        for x in xs:
            h = f(x, h)
            cum_results.append(h)
        return torch.stack(cum_results), h
    
    def tanh_cell(self, x, h, w_ih, w_hh, b_ih, b_hh):
        return torch.tanh(
            torch.einsum('ij,j', w_ih, x) + torch.einsum('ij,j', w_hh, h) + \
            b_ih + b_hh
        )
    
    def layers_forward(self, f, x, h_inits):
        '''forward pass in each layer
        returns last layers hidden variables and last hidden variable of each layer
        '''
        hs_L = []
        for i_layer, h_0 in enumerate(h_inits):
            f_partial = partial(f, w_ih=self.ws_ih[i_layer],
                                   w_hh=self.ws_hh[i_layer],
                                   b_ih=self.bs_ih[i_layer],
                                   b_hh=self.bs_hh[i_layer])
            x, h_L = self.cum_map(f_partial, x, h_0)
            hs_L.append(h_L)
        return x, torch.stack(hs_L)                

    def __call__(self, x, h_0=None):
        '''inputs   x:   [N, L, H_in]
                    h_0: [N, num_layers, H_out]
           outputs  h:   [N, L, H_out]
                    h_L  [N_layer, N, H_out]
        '''
        if h_0 is None:
            h_0 = torch.zeros(self.num_layers, N, self.hidden_size)
        h_0 = h_0.reshape(N, self.num_layers, self.hidden_size)
        
        batch_hs = []  # h sequence           [L, H_out]
        batch_hn = []  # last h in each layer [N_layer, H_out]
        # loop on each batch 
        for x_batch, h_0_batch in zip(x, h_0):
            hs, hn = self.batch_forward(x_batch, h_0_batch)
            batch_hs.append(hs)
            batch_hn.append(hn)
        return torch.stack(batch_hs), \
               torch.einsum('ijk->jik', torch.stack(batch_hn))  # N, N_layer, H_out -> N_layer, N, H_out

                

In [170]:
N = 2              # batch size
h_in = 2           # dimension of input vectors x_i
h_out = 5          # dimension of hidden variables h_i
num_layers = 3     # 
L = 100            # length of input sequence (x_1, ... x_L)


rnn_torch = nn.RNN(input_size=h_in, hidden_size=h_out, num_layers=num_layers, batch_first=True)
rnn_me = RNN(input_size=h_in, hidden_size=h_out, num_layers=num_layers)

x = torch.rand(N, L, h_in)                 # [N, L, h_in]
h_0 = torch.rand(num_layers, N, h_out)     # [num_layers, N, h_out]

First we copy all the parameters (weights and biases) from the torch rnn to the one we just made

In [171]:
from itertools import product

params = ['bias_ih_l', 'bias_hh_l', 'weight_ih_l', 'weight_hh_l']
for param, layer in product(params, [str(i) for i in range(num_layers)]):
    param_me = getattr(rnn_me, param+layer)
    param_torch = getattr(rnn_torch, param+layer)
    param_me.data = param_torch.data

In [172]:
hs_torch, hs_torch_n = rnn_torch(x)
hs_slow, hs_slow_n = rnn_me(x)

We can compare the results visually:

In [173]:
print(hs_torch[0, 0:10, :])
print(hs_slow[0, 0:10, :])

tensor([[-0.1588, -0.6647, -0.2076,  0.2584,  0.5815],
        [-0.0107, -0.8001,  0.0806, -0.2116,  0.3661],
        [-0.0609, -0.7705,  0.2482, -0.0761,  0.3675],
        [-0.0202, -0.7911,  0.2288,  0.0182,  0.3550],
        [-0.1055, -0.8029,  0.2264,  0.0278,  0.4159],
        [-0.0102, -0.8006,  0.2319,  0.0255,  0.3760],
        [-0.0800, -0.8151,  0.2640, -0.0154,  0.4040],
        [-0.1028, -0.7923,  0.2608,  0.1169,  0.4254],
        [-0.0493, -0.8170,  0.2063, -0.0154,  0.3472],
        [-0.0312, -0.7947,  0.2332,  0.0149,  0.4130]],
       grad_fn=<SliceBackward>)
tensor([[-0.1588, -0.6647, -0.2076,  0.2584,  0.5815],
        [-0.0107, -0.8001,  0.0806, -0.2116,  0.3661],
        [-0.0609, -0.7705,  0.2482, -0.0761,  0.3675],
        [-0.0202, -0.7911,  0.2288,  0.0182,  0.3550],
        [-0.1055, -0.8029,  0.2264,  0.0278,  0.4159],
        [-0.0102, -0.8006,  0.2319,  0.0255,  0.3760],
        [-0.0800, -0.8151,  0.2640, -0.0154,  0.4040],
        [-0.1028, -0.7923,  0.26

Or use the `norm` function for a quantitative difference of the two tensors: 

In [208]:
norm_diff = lambda x, y: torch.norm(x/torch.norm(x) - y/torch.norm(y))
norm_diff(hs_torch, hs_slow)
# torch.norm(hs_torch/torch.norm(hs_torch) - hs_slow/torch.norm(hs_slow))

tensor(1.7207e-07, grad_fn=<NormBackward0>)

which is negligible compared to the size of two tensors' entries.

To make sure that our implementation is according to the specification, we have to make sure that our the produced computatioanl graph for outputs are comparable to the one from pytorch. This is a hard problem with what I know of. In a sense one has to show that both graphs are, loosely speaking, **equivalent** to the canonical representation of the graph of the tensor. We may, however, seek to show that the gradients computed with `.backward` method are close enough to each other for a given function:

In [177]:
hs_torch, hs_torch_n = rnn_torch(x)
hs_slow, hs_slow_n = rnn_me(x)

# define a functional from output of both RNNs
l = torch.norm(torch.sum(hs_torch, dim=1))
rnn_torch.zero_grad()
l.backward()

l = torch.norm(torch.sum(hs_slow, dim=1))
rnn_me.zero_grad()
l.backward()

Next we loop over all the parameters of two model and compute the norm of the difference of their gradients

In [211]:
sum_norm_diff = 0.0
for param_me, param_torch in zip(rnn_me.parameters(), rnn_torch.parameters()):
    sum_norm_diff += norm_diff(param_me.grad, param_torch.grad)
print(sum_norm_diff)

tensor(2.4588e-06)


Which shows again a very good agreement between the two implementations, given that the used functional is by no means trivial ($l=\lVert \Sigma_{i}  h_i\rVert$).

Timing of the two implementations shows pytorch is 4 to 8 times faster than our implementation

In [212]:
%timeit hs, hs_n = rnn_me(x)

105 ms ± 766 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [213]:
%timeit hs, hs_n = rnn_torch(x)

12.6 ms ± 84.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
